In [1]:
!python3.9 -m pip install --user scikit-misc
!python3.9 -m pip install numpy==1.22.0

     |████████████████████████████████| 10.8 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 9.4 MB/s eta 0:00:011
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.1 requires numpy<1.23.0,>=1.16.5, but you have numpy 2.0.2 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
     |████████████████████████████████| 16.8 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [2]:
random_state=1000

In [3]:
# path1 = '/groups/NovaSeq-02/bioinformatics/users/project_prisyn/Human_TFs_Lambert.txt'
# path2 = '/groups/NovaSeq-02/bioinformatics/users/project_prisyn/Human_TFs_AnimalTFDB.txt'

path1 = '../data/raw/Human_TFs_Lambert.txt'
path2 = '../data/raw/Homo_sapiens_TF.csv'

with open(path1, "r") as f1:
    Human_TFs_Lambert = [line.strip() for line in f1]
    
Human_TFs_AnimalTFDB = []
with open(path2, "r") as f:

    next(f) 
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 3:
            ensembl_id = parts[1]
            if ensembl_id:  
                Human_TFs_AnimalTFDB.append(ensembl_id)

print(len(Human_TFs_AnimalTFDB))
print(len(Human_TFs_Lambert)) 

1665
1639


In [4]:
Human_TFs_AnimalTFDB_set = set(Human_TFs_AnimalTFDB)
Human_TFs_Lambert_set = set(Human_TFs_Lambert)

common_genes = Human_TFs_AnimalTFDB_set.intersection(Human_TFs_Lambert_set)
print(f"Found {len(common_genes)} common genes")

Found 1508 common genes


In [5]:
not_common_genes = Human_TFs_AnimalTFDB_set.symmetric_difference(Human_TFs_Lambert_set)
print(f"Found {len(not_common_genes)} genes not common in both sets")

Found 288 genes not common in both sets


In [6]:
unique_to_AnimalTFDB = Human_TFs_AnimalTFDB_set - Human_TFs_Lambert_set
unique_to_Lambert = Human_TFs_Lambert_set - Human_TFs_AnimalTFDB_set

print(f"Unique to AnimalTFDB: {(unique_to_AnimalTFDB)} genes")
print(f"Unique to Lambert: {(unique_to_Lambert)} genes")

Unique to AnimalTFDB: {'GON4L', 'CDIP1', 'ZNF559-ZNF177', 'SMAD2', 'CCDC88A', 'MTA2', 'DMRTC1B', 'PLEK', 'TAX1BP1', 'AC005324.4', 'SSRP1', 'CNBP', 'GABPB2', 'AC010616.1', 'BCLAF1', 'PMS1', 'KIAA1549', 'NCOR1', 'MAEL', 'HMGB1', 'NCOR2', 'AC105001.2', 'PIAS2', 'PIAS4', 'GATAD1', 'LITAF', 'CSDC2', 'RFXANK', 'AC004080.3', 'CREBRF', 'RERE', 'HMGB4', 'MAF1', 'CAPN15', 'JARID2', 'ARID1B', 'DUXB', 'SMAD6', 'TULP3', 'UBTFL1', 'TADA2A', 'MIER1', 'ZMIZ1', 'PAXBP1', 'MIER3', 'PIAS1', 'TFAM', 'TULP1', 'HOPX', 'TULP2', 'RCOR2', 'ZNF622', 'AL033529.1', 'CBFB', 'SMAD7', 'HMGB3', 'HMGXB4', 'AC187653.1', 'RCOR1', 'AC010487.3', 'AC073111.5', 'AC073612.1', 'AC067968.1', 'AC020909.1', 'KMT2C', 'NR0B2', 'ARID4B', 'MYEF2', 'TSC22D2', 'ZNFX1', 'AC092329.3', 'AC019117.3', 'AFF2', 'DNAJC1', 'ARID1A', 'HMGB2', 'PRDM11', 'WDHD1', 'AFF3', 'TOX', 'ID4', 'TBXT', 'AC115220.1', 'ELMSAN1', 'LRRFIP1', 'KLF18', 'AC073111.3', 'AC022167.5', 'ZMIZ2', 'SFPQ', 'ID2', 'SUB1', 'AC010422.6', 'TCF25', 'TSC22D3', 'DMRTC1', 'AC0252

## COVID dataset - haniffa 

In [7]:
import scanpy as sc 
import numpy as np 
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from sklearn.preprocessing import LabelEncoder

In [8]:
def load_adata(file_path):
    adata = sc.read_h5ad(file_path)
    return adata 

def map_disease_labels(adata, label_column, encoder_column): 
    encoder = LabelEncoder()  # Create a single instance of LabelEncoder 
    adata.obs[encoder_column] = encoder.fit_transform(adata.obs[label_column].values)  # Transform labels 
    mapping_disease_label = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store mapping 
    return adata, mapping_disease_label 

In [9]:
ls ../data/raw/COVID_Haniffa21/

haniffa21.max_patient.h5ad  haniffa21.processed.h5ad  preprocessing_covid.ipynb


In [10]:
# path = r"/groups/NovaSeq-02/bioinformatics/users/project_prisyn/haniffa_covid.h5ad" 
path = r"../data/raw/COVID_Haniffa21/haniffa21.processed.h5ad" 
adata_covid = load_adata(path) 

In [11]:
gene_mask = adata_covid.var['feature_types'] == 'Gene Expression' 
adata_rna = adata_covid[:, gene_mask] 
adata_covid_binary = adata_rna[adata_rna.obs['Status'].isin(['Covid', 'Healthy'])] 

In [12]:
adata_covid_binary

View of AnnData object with n_obs × n_vars = 624325 × 24737
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [13]:
del adata_covid_binary.obsm 
del adata_covid_binary.uns  

In [14]:
adata_covid_binary

AnnData object with n_obs × n_vars = 624325 × 24737
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    layers: 'raw'

In [15]:
adata_covid_binary, mapping_disease_label_covid_binary = map_disease_labels(adata=adata_covid_binary, 
                                                                            label_column='Status', 
                                                                            encoder_column='disease_label_binary') 

In [16]:
print(adata_covid_binary.obs.groupby('disease_label_binary', observed=True)['patient_id'].nunique())

disease_label_binary
0    90
1    23
Name: patient_id, dtype: int64


In [17]:
adata_covid_binary.layers['preprocessed']= adata_covid_binary.X.copy()  # Save the preprocessed data in a new layer 
adata_covid_binary.layers['raw_counts'] = adata_covid_binary.layers['raw'].copy() # saving raw counts and using it for  preprocessing 
adata_covid_binary.X = adata_covid_binary.layers['raw_counts'].copy() 
adata_covid_binary.var["mt"] = adata_covid_binary.var_names.str.startswith("MT-") 
adata_covid_binary.var["ribo"] = adata_covid_binary.var_names.str.startswith(("RPS", "RPL")) 
adata_covid_binary.var["hb"] = adata_covid_binary.var_names.str.contains("^HB[^(P)]") 
sc.pp.calculate_qc_metrics(adata_covid_binary, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=False) 
sc.pp.filter_cells(adata_covid_binary, min_genes=200) 
sc.pp.filter_genes(adata_covid_binary, min_cells=3) 

In [18]:
patient_ids = adata_covid_binary.obs['patient_id'].unique()
adata_list = [adata_covid_binary[adata_covid_binary.obs['patient_id'] == pid].copy() for pid in patient_ids] 

In [19]:
# For each patient, get set of expressed genes (expressed in ≥1 cell)
expressed_genes_per_patient = []
for ad in adata_list:
    if hasattr(ad.X, "toarray"):
        expr = ad.X.toarray()
    else:
        expr = ad.X 
    gene_sum = expr.sum(axis=0)
    genes_expressed = set(ad.var_names[gene_sum > 0])
    expressed_genes_per_patient.append(genes_expressed)

# Intersection across all patients
common_genes = set.intersection(*expressed_genes_per_patient)
print(f"Number of genes expressed in all patients: {len(common_genes)}")

for i, ad in enumerate(adata_list):
    genes_to_keep = ad.var_names.isin(common_genes)
    adata_list[i] = ad[:, genes_to_keep].copy() 

Number of genes expressed in all patients: 12087


In [20]:
adata_list_filtered = [ad for ad in adata_list if ad.n_obs >= 1000]

In [21]:
# pip install --user scikit-misc

In [22]:
for adata in adata_list_filtered:
    sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)
    sc.pp.log1p(adata) 
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat_v3', layer='raw_counts') 

In [23]:
1e4

10000.0

LINC00115      True
NOC2L         False
KLHL17        False
PLEKHN1       False
HES4           True
              ...  
MT-ND4        False
MT-ND5        False
MT-ND6         True
MT-CYB        False
AC240274.1    False
Name: highly_variable, Length: 12087, dtype: bool

In [38]:
for i in adata_list_filtered:
    print(i.X[:, i.var['highly_variable']].shape)

(8205, 2000)
(5655, 2000)
(10295, 2000)
(10446, 2000)
(9019, 2000)
(7128, 2000)
(10207, 2000)
(6847, 2000)
(1190, 2000)
(6149, 2000)
(6029, 2000)
(8819, 2000)
(3261, 2000)
(7585, 2000)
(7985, 2000)
(6808, 2000)
(10082, 2000)
(11710, 2000)
(10084, 2000)
(12081, 2000)
(8004, 2000)
(5505, 2000)
(2096, 2000)
(7816, 2000)
(5096, 2000)
(9608, 2000)
(9542, 2000)
(8232, 2000)
(10014, 2000)
(8244, 2000)
(7605, 2000)
(6388, 2000)
(6614, 2000)
(7300, 2000)
(7953, 2000)
(7100, 2000)
(7010, 2000)
(7016, 2000)
(2223, 2000)
(3217, 2000)
(8530, 2000)
(4316, 2000)
(9083, 2000)
(8111, 2000)
(5848, 2000)
(8639, 2000)
(14317, 2000)
(7011, 2000)
(8182, 2000)
(8371, 2000)
(8919, 2000)
(6932, 2000)
(7987, 2000)
(9458, 2000)
(6896, 2000)
(5589, 2000)
(3101, 2000)
(14086, 2000)
(8574, 2000)
(4494, 2000)
(3616, 2000)
(4760, 2000)
(7446, 2000)
(3815, 2000)
(10921, 2000)
(8511, 2000)
(6853, 2000)
(2727, 2000)
(3884, 2000)
(4951, 2000)
(2626, 2000)
(3219, 2000)
(5731, 2000)
(2233, 2000)
(1522, 2000)
(1184, 2000)
(

In [25]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
max_adata = max(adata_list_filtered, key=lambda ad: ad.n_obs) # all genes 
max_adata_hvg = max_adata[:, max_adata.var['highly_variable']].copy() # 2000 HVG 

In [ ]:
max_adata, max_adata_hvg

In [ ]:
def count_zero_expression_cells_per_gene(adata):
    if hasattr(adata.X, "toarray"):
        expr_matrix = adata.X.toarray()
    else:
        expr_matrix = adata.X
    zero_counts = (expr_matrix == 0).sum(axis=0)  # zero counts per gene
    zero_counts_per_gene = dict(zip(adata.var_names, zero_counts))
    return zero_counts_per_gene

zero_counts_per_gene = count_zero_expression_cells_per_gene(max_adata_hvg)
for gene, zero_count in zero_counts_per_gene.items():
    if zero_count > 0:
        print(f"{gene}: {zero_count} cells with zero expression") 

In [ ]:
sc.pl.highly_variable_genes(max_adata) 

In [ ]:
sc.tl.pca(max_adata_hvg)

In [ ]:
sc.pl.pca_variance_ratio(max_adata_hvg, n_pcs=50, log=True)

In [ ]:
sc.pl.pca(max_adata_hvg, color=["initial_clustering"])

In [ ]:
sc.pp.neighbors(max_adata_hvg) 

In [ ]:
sc.tl.umap(max_adata_hvg)

In [ ]:
sc.pl.umap(max_adata_hvg,color="initial_clustering") 

In [ ]:
sc.tl.leiden(max_adata_hvg)

In [ ]:
sc.pl.umap(max_adata_hvg, color=["leiden"])

In [ ]:
def stratified_split_adata(adata, label_key='initial_clustering', test_size=1000, val_size=1000, random_state=42):
    """Split AnnData into train, val, and test with stratification on a label column."""
    labels = adata.obs[label_key].values
    all_idx = np.arange(adata.n_obs)

    sss_test = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_val_idx, test_idx = next(sss_test.split(all_idx, labels))

    sss_val = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=random_state)
    train_idx, val_idx = next(sss_val.split(train_val_idx, labels[train_val_idx]))
    
    split = np.array(['train'] * adata.n_obs)
    split[test_idx] = 'test'
    split[train_val_idx[val_idx]] = 'val'
    adata.obs['split'] = split
    return adata 

In [ ]:
max_adata_hvg.shape

In [ ]:
max_adata_hvg = stratified_split_adata(max_adata_hvg, random_state=random_state)

# To access subsets:
max_adata_hvg_train = max_adata_hvg[max_adata_hvg.obs['split'] == 'train'].copy()
max_adata_hvg_val   = max_adata_hvg[max_adata_hvg.obs['split'] == 'val'].copy()
max_adata_hvg_test  = max_adata_hvg[max_adata_hvg.obs['split'] == 'test'].copy()

In [ ]:
for adata in [max_adata_hvg_train, max_adata_hvg_val, max_adata_hvg_test]:
    sc.pp.pca(adata, n_comps=50)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.pl.umap(adata,color="initial_clustering")   

In [ ]:
def compare_celltype_proportions(adata, split_key='split', label_key='initial_clustering', normalize=False):
    crosstab = pd.crosstab(adata.obs[label_key], adata.obs[split_key])
    if normalize:
        proportions = crosstab.div(crosstab.sum(axis=0), axis=1)
        return proportions
    else:
        return crosstab 

In [ ]:
import pandas as pd
# Get proportions of cell types across train/val/test
proportions_df = compare_celltype_proportions(max_adata_hvg)
print(proportions_df)
proportions_df.plot(kind='bar', figsize=(10, 5), title="Cell Type Proportions Across Splits")

In [ ]:
max_adata_hvg = stratified_split_adata(max_adata_hvg, test_size=2000, val_size=2000) #increasing the test and val size to 2000

# To access subsets:
max_adata_hvg_train_2000 = max_adata_hvg[max_adata_hvg.obs['split'] == 'train'].copy()
max_adata_hvg_val_2000   = max_adata_hvg[max_adata_hvg.obs['split'] == 'val'].copy()
max_adata_hvg_test_2000  = max_adata_hvg[max_adata_hvg.obs['split'] == 'test'].copy()

In [ ]:
proportions_df = compare_celltype_proportions(max_adata_hvg)
print(proportions_df)
proportions_df.plot(kind='bar', figsize=(10, 5), title="Cell Type Proportions Across Splits")

In [ ]:
# subsetting TFs 

# only for one donor from 2000 genes # Lambert 
hvg_genes = set(max_adata_hvg.var_names)
tfs_in_hvgs = Human_TFs_Lambert_set & hvg_genes # Get intersection of HVGs and TFs
print(f"Number of TFs found in HVGs: {len(tfs_in_hvgs)}") 

In [ ]:
max_adata_hvg

In [ ]:
def count_zero_expression_cells_per_gene(adata):
    if hasattr(adata.X, "toarray"):
        expr_matrix = adata.X.toarray()
    else:
        expr_matrix = adata.X
    zero_counts = (expr_matrix == 0).sum(axis=0)  # zero counts per gene
    zero_counts_per_gene = dict(zip(adata.var_names, zero_counts))
    return zero_counts_per_gene

zero_counts_per_gene = count_zero_expression_cells_per_gene(max_adata_hvg)
for gene, zero_count in zero_counts_per_gene.items():
    if zero_count > 0:
        print(f"{gene}: {zero_count} cells with zero expression") 

In [ ]:
def random_split_adata(adata, test_size=1000, val_size=1000, random_state=42):
    """Randomly split AnnData into train, val, and test (no stratification)."""
    all_idx = np.arange(adata.n_obs)

    # Split off test set
    ss_test = ShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_val_idx, test_idx = next(ss_test.split(all_idx))

    # Split remaining into train/val
    ss_val = ShuffleSplit(n_splits=1, test_size=val_size, random_state=random_state)
    train_idx, val_idx = next(ss_val.split(train_val_idx))

    # Initialize split labels
    split = np.array(['train'] * adata.n_obs)
    split[test_idx] = 'test'
    split[train_val_idx[val_idx]] = 'val'

    adata.obs['split'] = split
    return adata

In [ ]:
max_adata_hvg_random = random_split_adata(max_adata_hvg, random_state=random_state)

# To access subsets:
max_adata_hvg_train_random = max_adata_hvg[max_adata_hvg.obs['split'] == 'train'].copy()
max_adata_hvg_val_random   = max_adata_hvg[max_adata_hvg.obs['split'] == 'val'].copy()
max_adata_hvg_test_random = max_adata_hvg[max_adata_hvg.obs['split'] == 'test'].copy()

In [ ]:
for adata in [max_adata_hvg_train_random, max_adata_hvg_val_random, max_adata_hvg_test_random]:
    sc.pp.pca(adata, n_comps=50)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.pl.umap(adata,color="initial_clustering")   

In [ ]:
proportions_df = compare_celltype_proportions(max_adata_hvg_random)
print(proportions_df)
proportions_df.plot(kind='bar', figsize=(10, 5), title="Cell Type Proportions Across Splits")

In [ ]:
ls ../data/processed/COVID_Haniffa21/

In [ ]:
path=f'../data/processed/COVID_Haniffa21/CrossVal_{random_state}'
import os
os.makedirs(path)

In [ ]:
max_adata_hvg_train_2000.write_h5ad(f'{path}/COVID_Haniffa21_train.h5ad')
max_adata_hvg_test_2000.write_h5ad(f'{path}/COVID_Haniffa21_test.h5ad')
max_adata_hvg_val_2000.write_h5ad(f'{path}/COVID_Haniffa21_validation.h5ad')

In [ ]:
import scanpy as sc
random_state=1000
path=f'../data/processed/COVID_Haniffa21/CrossVal_{random_state}'
max_adata_hvg_train_2000 = sc.read_h5ad(f'{path}/COVID_Haniffa21_train.h5ad')
max_adata_hvg_test_2000 = sc.read_h5ad(f'{path}/COVID_Haniffa21_test.h5ad')
max_adata_hvg_val_2000 = sc.read_h5ad(f'{path}/COVID_Haniffa21_validation.h5ad')

In [ ]:
len(max_adata_hvg_test_2000)

In [ ]:
import numpy as np
def get_genes_with_all_zeros(adata):
    X_dense = adata.X.todense() if hasattr(adata.X, "todense") else adata.X

    # Find columns where all entries are zero
    all_zero_mask = np.all(X_dense == 0.0, axis=0).tolist()[0]

    # Get corresponding gene names
    all_zero_genes = adata.to_df().columns[all_zero_mask].tolist()

    return all_zero_genes

In [ ]:
get_genes_with_all_zeros(max_adata_hvg_train_2000)

In [ ]:
get_genes_with_all_zeros(max_adata_hvg_test_2000)

In [ ]:
get_genes_with_all_zeros(max_adata_hvg_val_2000)

In [ ]:
max_adata_split_dict={'train': max_adata_hvg_train_2000, 
              'test': max_adata_hvg_test_2000,
              'validation': max_adata_hvg_val_2000}

for split, max_adata in max_adata_split_dict.items():
    zero_expressd_genes = (((max_adata.X.todense()==0).sum(axis=0))==max_adata.shape[0]).sum()
    print(f"{split}: {zero_expressd_genes} genes have zero expression in all cells")

In [ ]:
# for split in [max_adata_hvg_train, max_adata_hvg_test, max_adata_hvg_val]:
#     zero_expressd_gaenes = (((split.X.todense()==0).sum(axis=0))==split.shape[0]).sum()
#     print(zero_expressd_genes)

In [ ]:
# for split in [max_adata_hvg_train_random, max_adata_hvg_test_random, max_adata_hvg_val_random]:
#     zero_expressd_genes = (((split.X.todense()==0).sum(axis=0))==split.shape[0]).sum()
#     print(zero_expressd_genes)

In [ ]:
for split in ['train', 'test', 'validation']:
    max_adata_old = sc.read_h5ad(f'../data/processed/COVID_Haniffa21-old/CrossVal_1000/COVID_Haniffa21_{split}.h5ad')
    zero_expressd_genes = len(get_genes_with_all_zeros(max_adata_old))
    print(f"{split}: {zero_expressd_genes} genes have zero expression in all cells")

In [ ]:
max_adata_hvg_test_2000.obs['initial_clustering']

In [ ]:
max_adata_old.obs['initial_clustering']

In [ ]:
import torch
torch.

In [ ]:
max_adata_hvg_test_2000.obs['initial_clustering']

In [ ]:
max_adata_hvg_test_2000.obs['leiden']

In [ ]:
max_adata_hvg_test_2000